In [21]:
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

In [22]:
print(pd.__version__)
print("scikit-learn version:", sklearn.__version__)


1.5.3
scikit-learn version: 1.2.2


In [23]:
data = pd.read_csv('/content/drive/MyDrive/PKL-JTK-Eben/September/Datalearns247/datalearns_tags.csv')

In [24]:
data

,nid,title,topic_name,tag_name,field_tags_target_id,field_topic_target_id
0,12,Mengenal Machine Learning,Machine Learning,Article,11,120.0
1,63,21 Tahun Solusi247,NaN,Article,11,NaN
2,69,Mengenal Feature Selection dalam Machine Learning,Machine Learning,Article,11,120.0
3,69,Mengenal Feature Selection dalam Machine Learning,Feature Selection,Article,11,121.0
4,71,Feature Selection Menggunakan Scikit-learn,Pandas,Article,11,124.0
...,...,...,...,...,...,...
60,17,local class incompatible: stream classdesc ser...,Zeppelin,Knowledge Base,13,119.0
61,17,local class incompatible: stream classdesc ser...,Spark,Knowledge Base,13,83.0
62,13,Instalasi Hadoop 3.2.2 pada Windows 10 WSL,Hadoop,Student Work,14,123.0
63,37,Instalasi Apache Hive Pada Ubuntu,NaN,Student Work,14,NaN


In [25]:
data['topic_name'] = data['topic_name'].astype(str)
grouped_data = data.groupby('nid')['topic_name'].agg(', '.join).reset_index()
data = data.merge(grouped_data, on='nid', suffixes=('', '_grouped'))
data = data.drop_duplicates(subset='title').reset_index(drop=True)

In [26]:
data['features'] = data['title'].str.lower() +' '+ data['topic_name_grouped'].str.lower() +' '+ data['tag_name'].str.lower()

In [27]:
data

,nid,title,topic_name,tag_name,field_tags_target_id,field_topic_target_id,topic_name_grouped,features
0,12,Mengenal Machine Learning,Machine Learning,Article,11,120.0,Machine Learning,mengenal machine learning machine learning art...
1,63,21 Tahun Solusi247,nan,Article,11,NaN,"nan, nan","21 tahun solusi247 nan, nan article"
2,69,Mengenal Feature Selection dalam Machine Learning,Machine Learning,Article,11,120.0,"Machine Learning, Feature Selection",mengenal feature selection dalam machine learn...
3,71,Feature Selection Menggunakan Scikit-learn,Pandas,Article,11,124.0,"Pandas, Python, Feature Selection, Scikit-learn",feature selection menggunakan scikit-learn pan...
4,190,"Analisis Time Series, Definisi, Pola dan Algor...",Timeseries,Article,11,112.0,Timeseries,"analisis time series, definisi, pola dan algor..."
5,191,13 Python Package Populer untuk Analisis Time ...,Python,Article,11,85.0,"Python, Timeseries, Statsmodels, Pmdarima, Pro...",13 python package populer untuk analisis time ...
6,201,Deteksi Stasioneritas Dalam Time Series,Timeseries,Article,11,112.0,Timeseries,deteksi stasioneritas dalam time series timese...
7,202,"Perjalanan Artificial Neural Network, Dari Per...",AI,Article,11,109.0,"AI, ANN, Deep Learning","perjalanan artificial neural network, dari per..."
8,212,HGrid247 Data Engineering,HGridDE,Article,11,122.0,HGridDE,hgrid247 data engineering hgridde article
9,233,Yuk Mengenal Apache Spark,Spark,Article,11,83.0,Spark,yuk mengenal apache spark spark article


In [28]:
import string

def remove_punctuation(text):
    if isinstance(text, str):
        translator = str.maketrans('', '', string.punctuation)
        return text.translate(translator)
    else:
        return text


In [29]:
data['features'] = data['features'].apply(remove_punctuation)

In [30]:
data['features']

0     mengenal machine learning machine learning art...
1                    21 tahun solusi247 nan nan article
2     mengenal feature selection dalam machine learn...
3     feature selection menggunakan scikitlearn pand...
4     analisis time series definisi pola dan algorit...
5     13 python package populer untuk analisis time ...
6     deteksi stasioneritas dalam time series timese...
7     perjalanan artificial neural network dari perc...
8             hgrid247 data engineering hgridde article
9               yuk mengenal apache spark spark article
10    30 tools generative ai tahun 2023 ai generativ...
11    inovasi masa depan ide bisnis ai untuk startup...
12                  hadoop benchmarking hadoop tutorial
13    penggunaan user impersonation pada apache zepp...
14    instalasi git bash pada windows git git bash t...
15    fitur reverse engineering pada hgrid247 de hgr...
16    belajar pyspark  select filter dan where pada ...
17    belajar apache spark dengan python  mengen

In [31]:
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform(data['features'])


In [32]:
similarity_scores = cosine_similarity(feature_matrix)

In [33]:
movie = data

In [34]:
def recommend_items_by_item_id(item_id, num_recommendations=7):
    item_index = data[data['nid'] == item_id].index[0]
    item_similarity = similarity_scores[item_index]
    similar_item_indices = item_similarity.argsort()[::-1][1:num_recommendations+1]
    recommended_item_ids = data.iloc[similar_item_indices]['nid'].values
    recommended_item_titles = data.iloc[similar_item_indices]['title'].values

    return recommended_item_ids, recommended_item_titles

In [35]:
item_id = 235
recommended_item_ids, recommended_item_titles = recommend_items_by_item_id(item_id)

selected_item = data[data['nid'] == item_id]['title'].values[0]

print(f"Title for ({item_id}): {selected_item}\n")

print(f"Recommended items for item ID {item_id}:")
for item_id, item_title in zip(recommended_item_ids, recommended_item_titles):
    print(f"Item ID: {item_id}, Title: {item_title}")


Title for (235): Belajar PySpark - Select, Filter dan Where Pada DataFrame

Recommended items for item ID 235:
Item ID: 245, Title: Belajar PySpark - SQL pada Dataframe dengan expr()
Item ID: 237, Title: Belajar PySpark - Transformasi DataFrame dengan withColumn
Item ID: 236, Title: Belajar Apache Spark Dengan Python : Mengenal DataFrame
Item ID: 238, Title: Belajar PySpark - Transformasi Dataframe dengan When-Otherwise
Item ID: 247, Title: Belajar PySpark - Join Dataframe
Item ID: 246, Title: Belajar PySpark - GroupBy dan Agregasi
Item ID: 233, Title: Yuk Mengenal Apache Spark


In [36]:
item_id = 12
recommended_item_ids, recommended_item_titles = recommend_items_by_item_id(item_id)

selected_item = data[data['nid'] == item_id]['title'].values[0]

print(f"Title for ({item_id}): {selected_item}\n")

print(f"Recommended items for item ID {item_id}:")
for item_id, item_title in zip(recommended_item_ids, recommended_item_titles):
    print(f"Item ID: {item_id}, Title: {item_title}")


Title for (12): Mengenal Machine Learning

Recommended items for item ID 12:
Item ID: 69, Title: Mengenal Feature Selection dalam Machine Learning
Item ID: 202, Title: Perjalanan Artificial Neural Network, Dari Perceptron ke Deep Learning
Item ID: 233, Title: Yuk Mengenal Apache Spark
Item ID: 236, Title: Belajar Apache Spark Dengan Python : Mengenal DataFrame
Item ID: 212, Title: HGrid247 Data Engineering
Item ID: 201, Title: Deteksi Stasioneritas Dalam Time Series
Item ID: 63, Title: 21 Tahun Solusi247


In [37]:
item_id = 191
recommended_item_ids, recommended_item_titles = recommend_items_by_item_id(item_id)

selected_item = data[data['nid'] == item_id]['title'].values[0]

print(f"Title for ({item_id}): {selected_item}\n")

print(f"Recommended items for item ID {item_id}:")
for item_id, item_title in zip(recommended_item_ids, recommended_item_titles):
    print(f"Item ID: {item_id}, Title: {item_title}")


Title for (191): 13 Python Package Populer untuk Analisis Time Series

Recommended items for item ID 191:
Item ID: 190, Title: Analisis Time Series, Definisi, Pola dan Algoritma
Item ID: 201, Title: Deteksi Stasioneritas Dalam Time Series
Item ID: 236, Title: Belajar Apache Spark Dengan Python : Mengenal DataFrame
Item ID: 263, Title: Inovasi Masa Depan: Ide Bisnis AI untuk Startup yang Menjanjikan
Item ID: 238, Title: Belajar PySpark - Transformasi Dataframe dengan When-Otherwise
Item ID: 237, Title: Belajar PySpark - Transformasi DataFrame dengan withColumn
Item ID: 245, Title: Belajar PySpark - SQL pada Dataframe dengan expr()


In [38]:
item_id = 202
recommended_item_ids, recommended_item_titles = recommend_items_by_item_id(item_id)

selected_item = data[data['nid'] == item_id]['title'].values[0]

print(f"Title for ({item_id}): {selected_item}\n")

print(f"Recommended items for item ID {item_id}:")
for item_id, item_title in zip(recommended_item_ids, recommended_item_titles):
    print(f"Item ID: {item_id}, Title: {item_title}")


Title for (202): Perjalanan Artificial Neural Network, Dari Perceptron ke Deep Learning

Recommended items for item ID 202:
Item ID: 12, Title: Mengenal Machine Learning
Item ID: 69, Title: Mengenal Feature Selection dalam Machine Learning
Item ID: 244, Title: 30 Tools Generative AI Tahun 2023
Item ID: 263, Title: Inovasi Masa Depan: Ide Bisnis AI untuk Startup yang Menjanjikan
Item ID: 233, Title: Yuk Mengenal Apache Spark
Item ID: 212, Title: HGrid247 Data Engineering
Item ID: 201, Title: Deteksi Stasioneritas Dalam Time Series


In [39]:
item_id = 212
recommended_item_ids, recommended_item_titles = recommend_items_by_item_id(item_id)

selected_item = data[data['nid'] == item_id]['title'].values[0]

print(f"Title for ({item_id}): {selected_item}\n")

print(f"Recommended items for item ID {item_id}:")
for item_id, item_title in zip(recommended_item_ids, recommended_item_titles):
    print(f"Item ID: {item_id}, Title: {item_title}")


Title for (212): HGrid247 Data Engineering

Recommended items for item ID 212:
Item ID: 221, Title: Fitur Reverse Engineering pada HGrid247 DE
Item ID: 233, Title: Yuk Mengenal Apache Spark
Item ID: 201, Title: Deteksi Stasioneritas Dalam Time Series
Item ID: 63, Title: 21 Tahun Solusi247
Item ID: 12, Title: Mengenal Machine Learning
Item ID: 190, Title: Analisis Time Series, Definisi, Pola dan Algoritma
Item ID: 263, Title: Inovasi Masa Depan: Ide Bisnis AI untuk Startup yang Menjanjikan
